In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23567.4"


In [ ]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [ ]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

In [ ]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

In [ ]:
#!connect azure-openai --model-type ChatCompletion --kernel-name chat --api-key @value:key --endpoint @value:endpoint --deployment gpt-35-turbo-16k --use-knowledge knowledge

# `#!graph` command

## define prompt template

In [ ]:
#!function generate_sequence_diagram --skill explain_code --description "generate mermaid sequence diagram from code"
[BEGIN TEXT]
{{$input}}
[END TEXT]
write mermaid sequence diagram markdown for the code above. remove all code fence markers

## test it 

In [ ]:
#!use-skills function.explain_code.generate_sequence_diagram

public static int Fibonacci(int n)
{
    if (n <= 1)
    {
        return n;
    }
    else
    {
        return Fibonacci(n - 1) + Fibonacci(n - 2);
    }
}



## integrate in `smart app`

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.AI;
using Microsoft.DotNet.Interactive.Mermaid;
using System.CommandLine;

var graphCommand = new Command("#!graph");
var textCompletionKernel = Kernel.Root.FindKernels(k => k is TextCompletionKernel).Single(t => t.Name == "chat(text)");
var mermaidKernel = Kernel.Root.FindKernels(k => k is MermaidKernel).Single();
IEnumerable<KernelEvent> evts = null;


Kernel.Root.AddDirective(graphCommand);

In [ ]:
evts.Display();

In [ ]:
var returnValueProduced = evts?
            .OfType<ReturnValueProduced>()
            .FirstOrDefault(e => e.Command.TargetKernelName == textCompletionKernel.Name);

             var markdown = returnValueProduced?
            .FormattedValues
            .OfType<FormattedValue>()
            .Single(fm => fm.MimeType == "text/plain")
            .Value
            .ToString() ?? string.Empty;

returnValueProduced.Display();

In [ ]:
public async Task GenerateDiagram(string code, KernelCommand root = null){
     textCompletionKernel.SetSuppressDisplay(true);
    List<KernelEvent> events = new();
    var prompt = $"""
        #!use-skills function.explain_code.generate_sequence_diagram

        {code}       
                
        """;
    evts = events;
    var sub = Kernel.Root.KernelEvents.Subscribe(e => events.Add(e));
    var createMermaidMarkdownCommand = new SubmitCode(prompt, textCompletionKernel.Name);
    if(root != null)
        createMermaidMarkdownCommand.SetParent(root);
    
    await textCompletionKernel.SendAsync(createMermaidMarkdownCommand);
    
    sub.Dispose();
    textCompletionKernel.SetSuppressDisplay(false);
    var returnValueProduced = events
        .OfType<ReturnValueProduced>()
        .FirstOrDefault(e => e.Command.TargetKernelName == textCompletionKernel.Name);

    var markdown = returnValueProduced?
        .FormattedValues
        .OfType<FormattedValue>()
        .Single(fm => fm.MimeType == "text/plain")
        .Value
        .ToString() ?? string.Empty;
 
    markdown = markdown.Replace("```mermaid", string.Empty).Replace("```", string.Empty);
    var renderMermaidMarkdownCommand = new SubmitCode(markdown, mermaidKernel.Name);
    if(root != null)
        renderMermaidMarkdownCommand.SetParent(root);
   
    await mermaidKernel.SendAsync(renderMermaidMarkdownCommand);
}

In [ ]:
await GenerateDiagram(
"""
public static int Fibonacci(int n)
{
    if (n <= 1)
    {
        return n;
    }
    else
    {
        return Fibonacci(n - 1) + Fibonacci(n - 2);
    }
}
""", KernelInvocationContext.Current.Command);

In [ ]:
graphCommand.SetHandler(async _ =>
{
    var context = KernelInvocationContext.Current;
    if (context.Command is SubmitCode submitCode)
    {       
        var code = submitCode.Code.Replace("#!graph", string.Empty);
        await GenerateDiagram(code,context.Command);
        context.Complete(submitCode);
    }
});

## try the app experience

In [ ]:
#!graph
public static int Fibonacci(int n)
{
    if (n <= 1)
    {
        return n;
    }
    else
    {
        return Fibonacci(n - 1) + Fibonacci(n - 2);
    }
}
